# Goal
* generate polymer-concatenated clean version of PDBBind *_protein.pdb 
* generate an entrycode-FASTA index file

In [33]:
from matplotlib import pyplot as plt 
from pathlib import Path
from tqdm import tqdm
import json

In [4]:
convert_residue = {
'ALA':'A', 'VAL':'V', 'PHE':'F', 'PRO':'P', 'MET':'M',
'ILE':'I', 'LEU':'L', 'ASP':'D', 'GLU':'E', 'LYS':'K',
'ARG':'R', 'SER':'S', 'THR':'T', 'TYR':'Y', 'HIS':'H',
'CYS':'C', 'ASN':'N', 'GLN':'Q', 'TRP':'W', 'GLY':'G',
'MSE':'M', 'ASX':'B', 'UNK' : 'X', 'SEC':'U','PYL':'O'
}

In [5]:
general_set_dir = Path('../pdbbind/v2016/general-set-except-refined')
refined_set_dir = Path('../pdbbind/v2016/refined-set')
set_dirs = [general_set_dir, refined_set_dir]

## Generating clean pdbs

### Assumptions 
Note that this depends on the following assumptions(loosely checked) on the PDBBind dataset,
* all non-polymer HETATM records are pushed at the end of the sequence section, omitting chainID information
* At the end of every polymer chain, there is a TER record
* Only one model
* No 'disorder'  

### Procedure

* Loop through ATOM/HETATM entries, while pushing the pair of (chainID, exact line) to a **queue**.
* When the queue has been nonempty, **assert** the newest chainID is identical to the last one
* Each time encountering TER, pop and append to a "list of output lines" 
* At EOL, **assert** that every remaining item in the queue has chainID ' '
* write the list of output lines to a file, while
  * skipping if the atom is 'H' 


In [6]:
l = [(0, 'a'), (0, 'b'), (0, 'c')]
print(all(a == 0 for a, _ in l))
print(all([]))

True
True


In [46]:
def write_clean_version(pdb_file, code):
    output_lines = []
    with open(pdb_file, 'r') as f:
        queue = []
        for line in f:
            if line.startswith('ATOM') or line.startswith('HETATM'):
                chainID = line[21]
                if queue != []:
                    last_chainID, _ = queue[-1]
                    if last_chainID != chainID:
                        raise Exception('chainID changed without TER:', str(pdb_file), line)
                queue.append((chainID, line))
            elif line.startswith('TER'):
                output_lines += [line for _, line in queue]
                queue = [] 
            else:
                continue
        if not all(chainID == ' ' for chainID, _ in queue):
            raise Exception('EOL without TER for an non-empty chainID:', str(pdb_file))
    clean_pdb_file = pdb_file.parent / f'{code}_clean.pdb'
    with open(clean_pdb_file, 'w') as f:
        for line in output_lines:
            if line[76:78].strip() != 'H':
                f.write(line)

In [16]:
def loop_over_data_dirs():
    for set_dir in set_dirs:
        print(set_dir)
        for data_dir in tqdm(list(set_dir.glob('*'))):
            if data_dir.name in ['readme', 'index']:
                continue
            yield data_dir

In [47]:
for data_dir in loop_over_data_dirs():
    protein_file = data_dir / f'{data_dir.name}_protein.pdb'    
    write_clean_version(protein_file, data_dir.name)
                    

../pdbbind/v2016/general-set-except-refined


100%|██████████████████████████████████████| 9228/9228 [00:45<00:00, 202.93it/s]


../pdbbind/v2016/refined-set


100%|██████████████████████████████████████| 4059/4059 [00:19<00:00, 209.45it/s]


## Checking whether the generated clean pdb files look find 
* Manually check the HETATM resnames 
* Ensuring that seqres records and residues coincide, with 70 exceptions 
* Every residue contains 'CA' (Actually, 571 exceptions)

### Resname checking procedure
* collect all hetcodes 
* print 

In [20]:
hetcodes = set([])
for data_dir in loop_over_data_dirs():
    clean_pdb_file = data_dir / f'{data_dir.name}_clean.pdb'
    with open(clean_pdb_file, 'r') as f:
        for line in f:
            if line.startswith('HETATM'):
                hetcode = line[17:20]
                hetcodes.add(hetcode) 
for hetcode in hetcodes:
    print(hetcode)

../pdbbind/v2016/general-set-except-refined


100%|██████████████████████████████████████| 9228/9228 [00:11<00:00, 810.17it/s]


../pdbbind/v2016/refined-set


100%|██████████████████████████████████████| 4059/4059 [00:04<00:00, 837.64it/s]


MSE
CSO
MLY
SEP
TPO
CSD
ACE
LLP
KCX
PCA
PTR


### Checking CA 
* Loop over lines:
  * When either 
    * you encounter a new residue or
    * EOL 
  * assert you have a CA atom in the previous residue 
* Record related statistics while doing that

In [49]:
problematic_resnames = set([])
num_problematic = 0
missing_cas_list = []
min_ca_proportion = 1.0 
max_missing_cas = 0
for data_dir in loop_over_data_dirs():
    clean_pdb_file = data_dir / f'{data_dir.name}_clean.pdb'
    num_residues = 0
    with open(clean_pdb_file, 'r') as f:
        chain = []
        chainID = None
        resSeq = None 
        iCode = None
        resname = None
        
        CA_encountered = True
        missing_cas = 0
        for line in f:
            prev_chainID = chainID 
            prev_resSeq = resSeq 
            prev_iCode = iCode
            prev_resname = resname
            
            chainID = line[21]
            resSeq = int(line[22:26])
            iCode = line[26]
            resname = line[17:20]
            atomname = line[12:16].strip()
            
            if (chainID, resSeq, iCode) != (prev_chainID, prev_resSeq, prev_iCode):
                num_residues += 1
                if not CA_encountered:
                    if not prev_resname in problematic_resnames:
                        missing_cas += 1
                        #print(prev_resname, clean_pdb_file.absolute(), prev_resSeq)
                        #problematic_resnames.add(prev_resname)
                    #raise Exception('No CA:', str(data_dir), prev_resSeq)
                CA_encountered = False
            if atomname == 'CA':
                if CA_encountered:
                    raise Exception('More than one CA:', str(data_dir), prev_resSeq)
                CA_encountered = True
        num_residues += 1 
        if not CA_encountered:
            missing_cas += 1
            #raise Exception('No CA:', str(data_dir), prev_resSeq)
        if missing_cas > 0:
            num_problematic += 1
            missing_cas_list.append(missing_cas) 
        if missing_cas > max_missing_cas:
            max_missing_cas = missing_cas 
        if 1 - missing_cas / num_residues < min_ca_proportion:
            min_ca_proportion = 1 - missing_cas / num_residues
print('# of cases at least one CA is missing:', num_problematic) #571
print('max missing cas:', max_missing_cas) #24
print('avg missing cas:', sum(missing_cas_list) / len(missing_cas_list)) #1.54
print('min proportion of ca-provided residues:', min_ca_proportion) #0.98

../pdbbind/v2016/general-set-except-refined


100%|██████████████████████████████████████| 9228/9228 [00:36<00:00, 251.81it/s]


../pdbbind/v2016/refined-set


100%|██████████████████████████████████████| 4059/4059 [00:15<00:00, 268.70it/s]

# of cases at least one CA is missing: 571
max missing cas: 24
min proportion of ca-provided residues: 0.9824561403508771


### SEQRES-ATOM/HETATM comparision procedure
* Parse seqres(list of resnames) from *_protein.pdb 
* Parse chains(list of resnames) from *_clean.pdb, based on 
  * only chainID for chain separation 
  * resSeq and iCode for residue 
* assert
    -The keys are identical 
    -lists from ATOM/HETATM are not longer than lists from SEQRES
    -If they have equal lengths, they are identical

In [22]:
def parse_seqres(file, fasta=True):
    seqs = {}
    with open(file, 'r') as reader:
        for line in reader:
            if not line.startswith('SEQRES'):
                continue
            chainID = line[11]
            if not chainID in seqs:
                seqs[chainID] = []
            subseq = [s.strip() for s in line[19:].split()]
            seqs[chainID] += subseq
        for chainID in seqs:
            if fasta:
                seqs[chainID] = ''.join([convert_residue.get(res, 'X') for res in seqs[chainID]])
    return seqs

In [32]:
num_shorter = 0
num_equal = 0
for data_dir in loop_over_data_dirs():
    protein_file = data_dir / f'{data_dir.name}_protein.pdb'
    clean_file = data_dir / f'{data_dir.name}_clean.pdb'
    
    seqres_dict = parse_seqres(protein_file, fasta=False)
    
    chain_dict = {}
    with open(clean_file, 'r') as f:
        chain = []
        chainID = None
        resSeq = None 
        iCode = None
        
        for line in f:
            prev_chainID = chainID 
            prev_resSeq = resSeq 
            prev_iCode = iCode 
            
            chainID = line[21]
            resSeq = int(line[22:26])
            iCode = line[26]
            resname = line[17:20]
            
            if chainID != prev_chainID and prev_chainID is not None:
                chain_dict[prev_chainID] = chain 
                chain = [] 
            if chainID != prev_chainID or (resSeq, iCode) != (prev_resSeq, prev_iCode):
                chain.append(resname) 
        chain_dict[chainID] = chain 
    
    if not set(seqres_dict.keys()) == set(chain_dict.keys()):
        raise Exception('SEQRES and ATOM/HETATM mismatch in chainIDS:', data_dir, set(seqres_dict.keys()), '!=', set(chain_dict.keys()))
    
    for chainID in seqres_dict.keys():
        seqres_chain = seqres_dict[chainID] 
        chain = chain_dict[chainID]
        
        if not len(seqres_chain) >= len(chain):
            Exception('SEQRES shorter than ATOM/HETATM:', data_dir)
        
        if len(seqres_chain) == len(chain):
            num_equal += 1
            if seqres_chain != chain:
                Exception('SEQRES not identical to ATOM/HETATM:', data_dir)
        else:
            num_shorter += 1
print('# of cases (ATOM/HETATM) <= SEQRES', num_shorter)
print('# of cases (ATOM/HETATM) == SEQRES', num_equal)

../pdbbind/v2016/general-set-except-refined


100%|██████████████████████████████████████| 9228/9228 [01:02<00:00, 148.14it/s]


../pdbbind/v2016/refined-set


100%|██████████████████████████████████████| 4059/4059 [00:26<00:00, 153.76it/s]

# of cases (ATOM/HETATM) <= SEQRES 69
# of cases (ATOM/HETATM) == SEQRES 22958


## Generating entrycode-FASTA index file

### file format 
Each line is f'{pdb_code}\t{FASTA}\n'
### procedure
* For each data_dir:
  * iterate lines of *_clean.pdb and collect resnames (concatenate all chains), based on change of (chainID, resSeq, iCode)
  * Turn it into FASTA using "convert_residue" 
  * put it into a pdb_code-FASTA dictinoary
* write the pdb_code-FASTA dictionary to <cc_fasta_dict.json>

In [51]:
cc_fasta_dict = {}
for data_dir in loop_over_data_dirs():
    clean_pdb = data_dir / f'{data_dir.name}_clean.pdb'
    with open(clean_pdb, 'r') as f:
        cc_chain = []
        chainID = None
        resSeq = None 
        iCode = None
        
        for line in f:
            prev_chainID = chainID 
            prev_resSeq = resSeq 
            prev_iCode = iCode 
            
            chainID = line[21]
            resSeq = int(line[22:26])
            iCode = line[26]
            resname = line[17:20]
            
            if chainID != prev_chainID or (resSeq, iCode) != (prev_resSeq, prev_iCode):
                cc_chain.append(resname) 
        
        cc_fasta = ''.join([convert_residue.get(res, 'X') for res in cc_chain])
        cc_fasta_dict[data_dir.name] = cc_fasta
        
with open('./cc_fasta_dict.json', 'w') as f:
    json.dump(cc_fasta_dict, f)
    

../pdbbind/v2016/general-set-except-refined


100%|██████████████████████████████████████| 9228/9228 [00:27<00:00, 335.35it/s]


../pdbbind/v2016/refined-set


100%|██████████████████████████████████████| 4059/4059 [00:11<00:00, 359.18it/s]
